# Impulsive BCR4BP problem

In [1]:
using DifferentialEquations
using Plots
using LinearAlgebra
import ForwardDiff
import DiffResults
using AstrodynamicsBase
import joptimise
using Printf

In [2]:
include("../../julia-r3bp/R3BP/src/R3BP.jl")
include("../src/SailorMoon.jl")   # relative path to main file of module

Main.SailorMoon

In [3]:
param3b = SailorMoon.dyanmics_parameters()
lps = SailorMoon.lagrange_points(param3b.mu2)

5×6 Matrix{Float64}:
  0.836915   0.0       0.0  0.0  0.0  0.0
  1.15568    0.0       0.0  0.0  0.0  0.0
 -1.00506    0.0       0.0  0.0  0.0  0.0
  0.487849   0.866025  0.0  0.0  0.0  0.0
  0.487849  -0.866025  0.0  0.0  0.0  0.0

In [4]:
lp = 2
Az_km = 1200.0
println("Halo guess Az_km: $Az_km")
northsouth = 3   # 1 or 3
guess0 = R3BP.halo_analytical_construct(param3b.mu2, lp, Az_km, param3b.lstar, northsouth)
res = R3BP.ssdc_periodic_xzplane([param3b.mu2,], guess0.x0, guess0.period, fix="period")
res.flag

Halo guess Az_km: 1200.0


1

In [5]:
x0_stm = vcat(res.x0, reshape(I(6), (6^2,)))[:]
prob_cr3bp_stm = ODEProblem(R3BP.rhs_cr3bp_svstm!, x0_stm, res.period, (param3b.mu2))
sol = solve(prob_cr3bp_stm, Tsit5(), reltol=1e-12, abstol=1e-12)#, saveat=LinRange(0, period, n+1))
monodromy = R3BP.get_stm(sol, 6)   # get monodromy matrix
ys0 = R3BP.get_eigenvector(monodromy, true, 1)
eps = 1e-6
if ys0[1] > 0
    svf = vcat(res.x0 + eps*ys0, 1.0)
else
    svf = vcat(res.x0 - eps*ys0, 1.0)
end

Linear stability ν = 618.7618470919092


7-element Vector{Float64}:
  1.1226882532375977
  2.793813746152613e-7
 -5.228883154550508e-26
 -7.710131455619898e-7
  0.16509880733691906
  1.339504158679477e-31
  1.0

In [6]:
# arrival LPO object
LPOArrival = SailorMoon.CR3BPLPO(
    res.x0, res.period, ys0, prob_cr3bp_stm, 1e-6, Tsit5(), 1e-12, 1e-12
);

### Construct optimization problem

In [7]:
LPOArrival.period

3.410360993933259

In [38]:
propagate_trajectory = function (x::AbstractVector{T}, get_sols::Bool=false) where T
    # unpack
    βf, tof, eta, sma, ecc, raan, ϕ = x  # βf: Sun angle at final time
    tof_fwd = tof * eta
    tof_bck = tof * (1 - eta)
    
    # construct initial state
    sv0_kep = [sma, ecc, 0.0, raan, 0.0, 0.0]
    β0 = βf - param3b.oms*(tof_fwd + tof_bck)   # initial Sun angle
    sv0_i = AstrodynamicsBase.kep2cart(sv0_kep, param3b.mu1)
    sv0 = vcat(inertial2rotating(sv0_i, β0, 1.0) + [-param3b.mu2,0,0,0,0,0], 1.0)
    
    # construct final state
    #x0_stm = vcat(LPOArrival.x0, reshape(I(6), (36,)))
    #tspan = [0, ϕ*LPOArrival.period]
    #prob_cr3bp_stm = ODEProblem(R3BP.rhs_cr3bp_svstm!, x0_stm, tspan, [param3b.mu2,])
    #sol = solve(prob_cr3bp_stm, Tsit5(), reltol=1e-12, abstol=1e-12)
    svf = vcat(SailorMoon.set_terminal_state(ϕ, param3b, LPOArrival), 1.0)
    
    # forward propagation
    params = [param3b.mu2, param3b.mus, β0, param3b.as, param3b.oms, 0.0, 0.0, 0.0, 0.0, 0.0]
    tspan = [0, tof_fwd]
    prob_fwd = ODEProblem(R3BP.rhs_bcr4bp_thrust!, sv0, tspan, params)
    sol_fwd = solve(prob_fwd, Tsit5(), reltol=1e-12, abstol=1e-12);
    
    # back propagation
    params = [param3b.mu2, param3b.mus, βf, param3b.as, param3b.oms, 0.0, 0.0, 0.0, 0.0, 0.0]
    tspan = [0, -tof_bck]
    prob_bck = ODEProblem(R3BP.rhs_bcr4bp_thrust!, svf, tspan, params)
    sol_bck = solve(prob_bck, Tsit5(), reltol=1e-12, abstol=1e-12);
    
    # residual
    if get_sols == false
        return sol_bck.u[end][1:3] - sol_fwd[end][1:3]
    else
        return sol_fwd, sol_bck
    end
end

#9 (generic function with 2 methods)

In [39]:
# θf, tof, eta, sma, ecc, raan = x
xtest = [
    2.4, 22, 0.4, 1.8, 0.87, 4.5, 0.02
]

lx = [
    2.6, 18, 0.3, 1.9, 0.7, 0.0, -1.0
]
ux = [
    3.2, 27, 0.7, 2.4, 0.995, 2π, 1.0
]

7-element Vector{Float64}:
  3.2
 27.0
  0.7
  2.4
  0.995
  6.283185307179586
  1.0

In [40]:
sol_fwd, sol_bck = propagate_trajectory(xtest, true);
propagate_trajectory(xtest, false)

3-element Vector{Float64}:
 -0.7592858619047174
 -0.052699171793091804
 -1.5272909535978123e-25

In [41]:
pcart = plot(size=(700,500), frame_style=:box, aspect_ratio=:equal, grid=0.4)
scatter!(pcart, lps[:,1], lps[:,2], marker=:diamond, color=:red, label="LPs")
# trajectory
plot!(Array(sol_fwd)[1,:], Array(sol_fwd)[2,:], color=:deeppink, linewidth=1.5, label="fwd")
plot!(Array(sol_bck)[1,:], Array(sol_bck)[2,:], color=:purple, linewidth=1.5, label="bck")

pcart

In [42]:
# problem settings
ng = 3
lg = [0.0 for idx=1:ng];
ug = [0.0 for idx=1:ng];

fitness! = function (g, x)
    # evaluate objective & objective gradient (trivial)
    #f = 1       # whichever x corresponds to e.g. mass at LEO
    #g[:] = propagate_trajectory(x, false)
    
    sol_fwd, sol_bck = propagate_trajectory(x, true)
    g[:] = sol_bck.u[end][1:3] - sol_fwd.u[end][1:3]
    f = norm(sol_bck.u[end][4:6] - sol_fwd.u[end][4:6])
    return f
end

#15 (generic function with 1 method)

In [43]:
gfoo  = zeros(ng)
fitness!(gfoo, xtest)
gfoo

3-element Vector{Float64}:
 -0.7592858619047174
 -0.052699171793091804
 -1.5272909535978123e-25

In [ ]:
ip_options = Dict(
    "max_iter" => 100,   # approx 100
    "print_level" => 4,
    "acceptable_tol" => 1e-6,
    "constr_viol_tol" => 1e-6,
)
x0 = [el for el in xtest]
xopt, fopt, info = joptimise.minimize(
    fitness!, x0, ng;
    lx=lx, ux=ux, lg=lg, ug=ug,
    #derivatives=joptimise.ForwardAD(),
    options=ip_options,
);

In [ ]:
info

In [16]:
sol_fwd, sol_bck = propagate_trajectory(xopt, true);
propagate_trajectory(xopt, false)

3-element Vector{Float64}:
 -3.655435878435753e-7
 -2.73251321125656e-7
 -6.410339451318033e-32

In [17]:
dr = sol_bck.u[end][1:3] - sol_fwd.u[end][1:3]

3-element Vector{Float64}:
 -3.655435878435753e-7
 -2.73251321125656e-7
 -6.410339451318033e-32

In [18]:
dv = sol_bck.u[end][4:6] - sol_fwd.u[end][4:6]

3-element Vector{Float64}:
  0.014089350044106386
 -0.10400386695822239
 -2.8775315140047104e-26

In [19]:
norm(dv)*param3b.lstar/param3b.tstar  # km/s

0.10748146721191036

In [20]:
for i = 1:length(xopt)
    @printf("% 3.3f | % 3.3f | % 3.3f\n", lx[i], xopt[i], ux[i])
end

 2.600 |  3.200 |  3.200
 18.000 |  22.413 |  27.000
 0.300 |  0.676 |  0.700
 1.900 |  2.252 |  2.400
 0.700 |  0.970 |  0.995
 0.000 |  6.283 |  6.283
-1.000 | -0.089 |  1.000


In [21]:
lx

7-element Vector{Float64}:
  2.6
 18.0
  0.3
  1.9
  0.7
  0.0
 -1.0

In [22]:
ux

7-element Vector{Float64}:
  3.2
 27.0
  0.7
  2.4
  0.995
  6.283185307179586
  1.0

In [23]:
plotly()

┌ Warning: For saving to png with the `Plotly` backend `PlotlyBase` and `PlotlyKaleido` need to be installed.
│   err = ArgumentError("Package PlotlyKaleido not found in current path:\n- Run `import Pkg; Pkg.add(\"PlotlyKaleido\")` to install the PlotlyKaleido package.\n")
└ @ Plots C:\Users\yshimane3\.julia\packages\Plots\fw4rv\src\backends.jl:426


Plots.PlotlyBackend()

In [24]:
pcart = plot(size=(700,500), frame_style=:box, aspect_ratio=:equal, grid=0.4)
scatter!(pcart, lps[:,1], lps[:,2], marker=:diamond, color=:red, label="LPs")
# trajectory
plot!(Array(sol_fwd)[1,:], Array(sol_fwd)[2,:], color=:deeppink, linewidth=1.5, label="fwd")
plot!(Array(sol_bck)[1,:], Array(sol_bck)[2,:], color=:purple, linewidth=1.5, label="bck")

pcart

In [25]:
# unpack optimal solution
βf, tof, eta, sma, ecc, raan, ϕ = xopt
tof_fwd = tof * eta
tof_bck = tof * (1 - eta)
β0 = βf - param3b.oms*tof                # initial Sun angle

23.93608527230465

In [26]:
tof

22.412543542652777

## Plot in Sun-Earth rotating frame

In [28]:
SE_system = R3BP.get_cr3bp_param(10,399)
EM_system = R3BP.get_cr3bp_param(399,301)

scale_l = EM_system.lstar/SE_system.lstar
scale_v = (EM_system.lstar/EM_system.tstar) / (SE_system.lstar/SE_system.tstar)
scale_t = EM_system.tstar/SE_system.tstar

0.07469828599591845

In [29]:
function planetmoon2sunplanet(
    state::Vector, μ_moon::Real, μ_sun::Real, theta::Real, om::Real,
    scale_l::Real, scale_v::Real, scale_t::Real, final_shift::Bool=false
)
    # 1. shift to Planet-centered, planet-moon rotating frame
    state_1 = state + [μ_moon, 0, 0, 0, 0, 0]
    # 2. convert to PLanet-centered, inertial
    state_2 = rotating2inertial(state_1, theta, om)
    # 3. re-scale
    state_3 = vcat(
        state_2[1:3] * scale_l,
        state_2[4:6] * scale_v
    )
    # 4. convert to Planet-centered, Sun-planet rotating frame
    theta_sp = scale_t * theta
    state_4 = inertial2rotating(state_3, theta_sp, om)
    # 5. shift to Sun-planet barycenter centered, Sun-planet rotating frame
    if final_shift == true
        return state_4 + [1 - μ_sun, 0, 0, 0, 0, 0]
    else
        return state_4
    end
end


planetmoon2sunplanet (generic function with 2 methods)

In [30]:
sol_bck_se = zeros(7,length(sol_bck.u))
for (idx,sv) in enumerate(sol_bck.u)
    theta = βf + sol_bck.t[idx]
    sol_bck_se[1:6,idx] = planetmoon2sunplanet(
        sv[1:6], param3b.mu2, param3b.mus, theta, 1.0, #param3b.oms,
        scale_l, scale_v, scale_t
    )
    sol_bck_se[7,idx] = sv[7]
end

sol_fwd_se = zeros(7,length(sol_fwd.u))
for (idx,sv) in enumerate(sol_fwd.u)
    theta = βf + sol_bck.t[end] - (tof_fwd - sol_fwd.t[idx])
    sol_fwd_se[1:6,idx] = planetmoon2sunplanet(
        sv[1:6], param3b.mu2, param3b.mus, theta, 1.0, #param3b.oms,
        scale_l, scale_v, scale_t
    )
    sol_fwd_se[7,idx] = sv[7]
end

In [31]:
pcart = plot(size=(700,500), frame_style=:box, aspect_ratio=:equal, grid=0.4)

# trajectory
#plot!(Array(sol_fwd)[1,:], Array(sol_fwd)[2,:], color=:deeppink, linewidth=1.5, label="fwd")
plot!(sol_fwd_se[1,:], sol_fwd_se[2,:], color=:deeppink, linewidth=1.5, label="fwd")
plot!(sol_bck_se[1,:], sol_bck_se[2,:], color=:purple, linewidth=1.5, label="bck")

pcart

## Plot in inertial frame

In [32]:
sol_fwd.t[end] - sol_bck.t[end]

22.412543542652777

In [36]:
sol_fwd_i = zeros(7,length(sol_fwd.u))
for (idx,sv) in enumerate(sol_fwd.u)
    theta = βf - tof + sol_fwd.t[idx]
    sol_fwd_i[1:6,idx] = rotating2inertial(sv[1:6], theta, 1.0)
    sol_fwd_i[7,idx] = sv[7]
end

sol_bck_i = zeros(7,length(sol_bck.u))
for (idx,sv) in enumerate(sol_bck.u)
    theta = βf + sol_bck.t[idx]
    sol_bck_i[1:6,idx] = rotating2inertial(sv[1:6], theta, 1.0)
    sol_bck_i[7,idx] = sv[7]
end

In [37]:
pcart = plot(size=(700,500), frame_style=:box, aspect_ratio=:equal, grid=0.4)

# trajectory
plot!(sol_fwd_i[1,:], sol_fwd_i[2,:], color=:deeppink, linewidth=1.5, label="fwd")
plot!(sol_bck_i[1,:], sol_bck_i[2,:], color=:purple, linewidth=1.5, label="bck")

pcart